Project Proposal (max 500 words)

Title

Introduction:
Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- heart disease affects x in y canadians
- it's very important heart disese, very painful & sadge
- talk about how hard to spot? 

Clearly state the question you will try to answer with your project
Identify and describe the dataset that will be used to answer the question
- we will try to classify a new patient on whether or not they may have heart disease based on other predictor variables

Preliminary exploratory data analysis:
Demonstrate that the dataset can be read from the web into R 
Clean and wrangle your data into a tidy format

- mutate the last column (assuming everything 1+ is 1 to follow their documentation)
- mutated last column to be factor
- mutated 2 columns to be numeric instead of character

Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- means of predictor variables
- number of rows with missing data

Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.
- use one plot to visualize distributions of predictor variables

Methods:
Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results

Expected outcomes and significance:
What do you expect to find?
What impact could such findings have?
What future questions could this lead to?

In [41]:
library(tidyverse)
library(tidymodels)

In [61]:
# nrow(cleveland_data) 303 rows
cleveland_data <- read_csv("data/processed.cleveland.data",
                          col_names = c("age", "sex", "chest_pain_type", "resting_blood_sugar", "cholestorol", "fasting_blood_sugar", "electrocardio_results", "max_heart_rate", "exercise_induced_angina", "ST_depression", "ST_peak_slope", "major_vessels", "thal", "diagnosis"))

# selects the diagnosis column and replaces all values > 1 with 1
# code inspired from https://datacornering.com/replace-r-data-frame-column-values-conditionally/
cleveland_data[c("diagnosis")][which(cleveland_data$diagnosis > 1), ] <- 1

# cleaning up columns to be the correct type
cleveland_data <- cleveland_data %>%
    mutate(major_vessels = as.numeric(major_vessels, na.rm = TRUE)) %>%
    mutate(thal = as.numeric(thal, na.rm = TRUE)) %>%
    mutate(diagnosis = as.factor(diagnosis)) 

# cleveland_data

knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 10) %>%
    set_engine("kknn") %>%
    set_mode("classification")
# knn_spec

knn_recipe <- recipe(diagnosis ~ ., data = cleveland_data) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())
# knn_recipe

knn_workflow <- workflow() %>%
    add_recipe(knn_recipe) %>%
    add_model(knn_spec) 
# knn_workflow

knn_fit <- knn_workflow %>%
    fit(data = cleveland_data)
# knn_fit

new_patient = tibble(age = 63, sex = 0, chest_pain_type = 3, resting_blood_sugar = 200, cholestorol = 300, fasting_blood_sugar = 0, electrocardio_results = 2, max_heart_rate = 180, exercise_induced_angina = 0, ST_depression = 0.9, ST_peak_slope = 3, major_vessels = 2, thal = 6)

prediction <- predict(knn_fit, new_patient)

# 0 = No Heart Disease, 1 = Heart Disease
prediction


Parsed with column specification:
cols(
  age = col_double(),
  sex = col_double(),
  chest_pain_type = col_double(),
  resting_blood_sugar = col_double(),
  cholestorol = col_double(),
  fasting_blood_sugar = col_double(),
  electrocardio_results = col_double(),
  max_heart_rate = col_double(),
  exercise_induced_angina = col_double(),
  ST_depression = col_double(),
  ST_peak_slope = col_double(),
  major_vessels = col_character(),
  thal = col_character(),
  diagnosis = col_double()
)

Warning message:
“Problem with `mutate()` input `major_vessels`.
ℹ NAs introduced by coercion
ℹ Input `major_vessels` is `as.numeric(major_vessels, na.rm = TRUE)`.”
Warning message in mask$eval_all_mutate(dots[[i]]):
“NAs introduced by coercion”
Warning message:
“Problem with `mutate()` input `thal`.
ℹ NAs introduced by coercion
ℹ Input `thal` is `as.numeric(thal, na.rm = TRUE)`.”
Warning message in mask$eval_all_mutate(dots[[i]]):
“NAs introduced by coercion”


.pred_class
<fct>
1
